In [2]:
!pip install dominate

In [3]:
from options.train_options import TrainOptions
from data import create_dataset
from models import create_model
from models.networks import arch_parameters
from util.visualizer import Visualizer
from util.util import print_timestamped
import time
import torch
from torch.utils.data import DataLoader, random_split, Subset

In [ ]:
if __name__ == '__main__':
    opt = TrainOptions().parse()  # get training options
    # Since the 3d training is very intense, we don't print
    if opt.model == "pix2pix3d":
        opt.display_id = -1
    data_loader = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
    dataset = data_loader.dataset
    total_size = len(dataset)
    split_1 = 98
    split_2 = total_size - split_1
    # Perform the split
    subset1, subset2 = random_split(dataset, [split_1, split_2])
    dataset = DataLoader(subset1, batch_size=opt.batch_size, num_workers=int(opt.num_threads), shuffle=not opt.serial_batches)
    
    dataset_size = len(dataset)  # get the number of images in the dataset.
    print('The number of training images = %d' % dataset_size)
    model = create_model(opt)  # create a model given opt.model and other options
    model.setup(opt)  # regular setup: load and print networks; create schedulers
    # Print network information
    if opt.print_model_info and "pix2pix" in opt.model:
        from torchsummary import summary
        final_size = opt.crop_size if "crop" in opt.preprocess else opt.load_size
        if "3d" in opt.model:
            summary(model.netG, (opt.input_nc, final_size, final_size, final_size))
        else:
            summary(model.netG, (opt.input_nc, final_size, final_size))
    visualizer = Visualizer(opt)  # create a visualizer that display/save images and plots
    total_iters = 0  # the total number of training iterations
    init_time = time.time()
    
    for epoch in range(opt.epoch_count,
                       opt.n_epochs + opt.n_epochs_decay + 1):  # outer loop for different epochs;
        # we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
        epoch_start_time = time.time()  # timer for entire epoch
        iter_data_time = time.time()  # timer for data loading per iteration
        epoch_iter = 0  # the number of training iterations in current epoch, reset to 0 every epoch
        visualizer.reset()  # reset the visualizer: make sure it saves the results to HTML at least once every epoch

        for i, data in enumerate(dataset):  # inner loop within one epoch
            iter_start_time = time.time()  # timer for computation per iteration
            if total_iters % opt.print_freq == 0:
                t_data = iter_start_time - iter_data_time

            total_iters += opt.batch_size
            epoch_iter += opt.batch_size
            model.set_input(data)  # unpack data from dataset and apply preprocessing
            model.optimize_parameters()  # calculate loss functions, get gradients, update network weights

            if total_iters % opt.display_freq == 0:  # display images on visdom and save images to a HTML file
                save_result = total_iters % opt.update_html_freq == 0
                model.compute_visuals()
                visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

            if total_iters % opt.print_freq == 0:  # print training losses and save logging information to the disk
                losses = model.get_current_losses()
                t_comp = (time.time() - iter_start_time) / opt.batch_size
                visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
                if opt.display_id > 0:
                    visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

            if total_iters % opt.save_latest_freq == 0:  # cache our latest model every <save_latest_freq> iterations
                print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
                save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
                model.save_networks(save_suffix)

            iter_data_time = time.time()
        model.update_learning_rate()  # update learning rates in the end of every epoch.
        if epoch % opt.save_epoch_freq == 0:  # cache our model every <save_epoch_freq> epochs
            print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
            model.save_networks('latest')
            model.save_networks(epoch)

        print('End of epoch %d / %d \t Time Taken: %d sec' % (
            epoch, opt.n_epochs + opt.n_epochs_decay, time.time() - epoch_start_time))
    torch.save(arch_parameters(), model.save_dir + "/arch_parameters.pth")
    end_time = round(time.time() - init_time, 3)
    print_timestamped("The training process took " + str(end_time) + "s.")